### 네이버 주식 일본의 코드 종목을 전부 가져오고 그 코드에 대한 10년치 데이터 수집

In [ ]:
def fetch_data(symbol):
    url = f"https://api.stock.naver.com/chart/foreign/item/{symbol}.T/day?startDateTime=201401150000&endDateTime=202401150000"
    response = requests.get(url).text
    data = json.loads(response)
    return symbol, pd.DataFrame(data)

all_data = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit fetch_data function for each symbol
    futures = [executor.submit(fetch_data, symbol) for symbol in symbol_codes]
    
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Fetching Data"):
        try:
            symbol, data_df = future.result()
            data_df['symbol'] = symbol
            all_data.append(data_df)

            output_folder = 'Japan_new'
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            
            csv_path = os.path.join(output_folder, f'{symbol}.csv')
            data_df.to_csv(csv_path, index=False)
            
        except Exception as e:
            print(f'{e}')

combined_df = pd.concat(all_data, ignore_index=True)

for df in tqdm(all_data, desc="Processing Data"):
    df['localDate'] = pd.to_datetime(df['localDate'], format='%Y%m%d')
    df.rename(columns={
        'closePrice': 'close',
        'openPrice': 'open',
        'highPrice': 'high',
        'lowPrice': 'low',
        'accumulatedTradingVolume': 'volume',
        'localDate': 'date'
    }, inplace=True)



### - startdate로 날짜를 지정해서 데이터 양을 조절했음.
### - 병렬 처리하는 함수를 사용하여 데이터 처리 속도를 높이려고 했음.
### - japan_new 라는 폴더안에 데이터를 저장하고 없으면 폴더를 생성.
### - 데이터 컬럼명을 변경해서 다른 사람과 컬럼명을 바춰 나중에 합치기 좋게 생성.

# 하루 치 데이터를 전체 종목을 가져와서 크론탭에 실행 계획 대로 실행하고 하둡에 업데이트 하는 코드


In [ ]:
import os
import pandas as pd
import requests
import json
from datetime import datetime
from tqdm import tqdm
import subprocess

HADOOP_HOME = "/home/hadoop/hadoop"
HDFS_PATH = "/stock/TYO"
LOCAL_DATA_PATH = "/home/hadoop/today_update"

def fetch_japan_stocks(url):
    all_data = []  

    page = 1
    while True:
        payload = {
            "page": str(page),
            "pageSize": "20"
        }
        response = requests.get(url, params=payload)
        if response.status_code == 200:
            data = response.json().get('stocks', [])
            all_data.extend(data)

            if len(data) == 0:
                break 
            else:
                page += 1  
        else:
            print(f'HTTP 요청 실패: {response.status_code}')
            break

    return pd.DataFrame(all_data)

def fetch_and_save_data(symbol_codes, output_folder='today_update'):
    current_date_str = datetime.now().strftime('%Y%m%d')

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for symbol in tqdm(symbol_codes, desc="Fetching Data"):
        url = f"https://api.stock.naver.com/chart/foreign/item/{symbol}.T/day?startDateTime={current_date_str}0000&endDateTime={current_date_str}2359"
        response = requests.get(url).text
        data = json.loads(response)

        data_df = pd.DataFrame(data)
        data_df['Symbol_Code'] = symbol
        data_df.rename(columns={
            'closePrice': 'close',
            'openPrice': 'open',
            'highPrice': 'high',
            'lowPrice': 'low',
            'accumulatedTradingVolume': 'volume',
            'localDate': 'date'
        }, inplace=True)

        csv_filename = f'TYO_{current_date_str}_{symbol}.csv'
        csv_path = os.path.join(output_folder, csv_filename)
        data_df.to_csv(csv_path, index=False)

def copy_data_to_hdfs():
    # Hadoop fs -put 명령어 실행
    cmd = [f"{HADOOP_HOME}/bin/hadoop", "fs", "-put", LOCAL_DATA_PATH, HDFS_PATH]

    try:
        subprocess.run(cmd, check=True)
        print("Data copied to HDFS successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error copying data to HDFS: {e}")

def main():
    url = "https://api.stock.naver.com/stock/exchange/TOKYO/"
    japan_stocks = fetch_japan_stocks(url)
    symbol_codes = japan_stocks['symbolCode'].tolist()

    fetch_and_save_data(symbol_codes)
    copy_data_to_hdfs()

    # today_update 폴더 삭제
    try:
        os.rmdir(LOCAL_DATA_PATH)
        print("today_update folder deleted successfully.")
    except OSError as e:
        print(f"Error deleting today_update folder: {e}")

if __name__ == "__main__":
    main()


# sh 실행하는 방법
- #!bin/bash 
- source ~/.bashrc
- python name.py 
- hdfs dfs -put 파일 경로(절대 경로) :/ 넣는 경로 


crontab 에서 스케줄링 걸어놓으면 실행되어서 하둡에 잘 들어감.
